<h1 align="center">Mobile Money Financial Inclusion Analyzer</h1>


In [40]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

In [17]:
df = pd.read_csv('C:/Users/USER/Group-10-Mobile-Money-Financial-Inclusion-Analyzer/data/2024_Finaccess_Publicdata.csv',)

C:\Users\USER\AppData\Local\Temp\ipykernel_25404\2124929324.py:1: DtypeWarning: Columns (30,32,59,62,86,96,110,132,147,162,177,237,252,277,279,386,393,395,420,422,461,480,482,484,536,554,567,578,582,584,586,588,597,599,601,614,630,638,640,649,657,659,661,663,672,674,676,678,687,689,700,702,704,715,717,719,721,732,736,745,747,749,758,760,762,771,773,775,784,786,790,798,800,802,810,818,820,822,832,859,902,904,921,922,923,924,925,926,927,928,929,930,949,950,951,952,954,955,966,978,980,982,984,986,992,994,996,998,1002,1004,1006,1008,1010,1012,1014,1016,1018,1020,1022,1024,1026,1028,1030,1032,1034,1038,1040,1042,1044,1046,1048,1050,1052,1065,1077,1086,1088,1090,1094,1096,1108,1119,1127,1128,1129,1130,1131,1132,1133,1134,1135,1136,1137,1138,1139,1140,1141,1142,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1155,1156,1157,1158,1159,1160,1162,1163,1164,1165,1166,1167,1168,1169,1170,1171,1172,1173,1174,1175,1176,1177,1178,1180,1181,1182,1183,1184,1185,1186,1187,1188,1189,1190,1191,1192,

In [18]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20871 entries, 0 to 20870
Columns: 3816 entries, interview__key to Longitude
dtypes: float64(44), int64(191), object(3581)
memory usage: 607.6+ MB


,interview__key,interview__id,county,A07,A08,A10,Interview_Status,A9i,Z1,A13,...,sacco_redress,mobilemoney_redress,mobilebank_redress,not_registered_mmoney_24,using_someone_acc,insurance_including_NHIF_use,All_Insurance_excluding_NHIF_use,PWD,Latitude,Longitude
0,21-61-43-44,00005e63e7424aa19e63adb071955233,Garissa,225,Urban,1,Household found with competent respondent,Access granted,completed,Male,...,#NULL!,#NULL!,#NULL!,No,#NULL!,Never used,Never used,Without Disability,-0.435423,39.636586
1,87-78-54-60,000718d26f5942998522033a66594b4c,Garissa,223,Urban,3,Household found with competent respondent,Access granted,completed,Male,...,#NULL!,#NULL!,#NULL!,No,#NULL!,Never used,Never used,Without Disability,0.058794,40.305006
2,32-71-19-84,0008721681d2468aa1ef456bba4475dc,Busia,1514,Urban,3,Household found with competent respondent,Access granted,completed,Female,...,#NULL!,#NULL!,#NULL!,No,#NULL!,Never used,Never used,Without Disability,0.636836,34.277390
3,48-18-78-99,00099e961d49470585c1a7f49fa3b7fb,Kiambu,796,Urban,3,Household found with competent respondent,Access granted,completed,Male,...,#NULL!,#NULL!,#NULL!,No,#NULL!,Never used,Never used,Without Disability,-1.251917,36.719076
4,50-82-98-89,000bb72e959b4dd48443da902c01e920,Murang'a,763,Urban,16,Household found with competent respondent,Access granted,completed,Female,...,#NULL!,#NULL!,#NULL!,No,Yes,Never used,Never used,Without Disability,-0.795820,37.131085


In [19]:
df = df[['county', 'A08', 'A13', 'A15a', 'A14iv_e', 'B3D__2_1', 'B3D__3_1']]
df.head()

,county,A08,A13,A15a,A14iv_e,B3D__2_1,B3D__3_1
0,Garissa,Urban,Male,English,"""Post Secondary education",#NULL!,#NULL!
1,Garissa,Urban,Male,English,NaN,#NULL!,#NULL!
2,Busia,Urban,Female,Swahili,Upper primary,#NULL!,#NULL!
3,Kiambu,Urban,Male,English,"""Secondary/High school",#NULL!,#NULL!
4,Murang'a,Urban,Female,Swahili,"""Post Secondary education",#NULL!,#NULL!


In [32]:
df = df.rename(columns={
    'A08': 'Rural/Urban',
    'A13': 'Gender',
    'A15a': 'Language',
    'A14iv_e': 'Education',
    'B3D__2_1': 'Mobile Money: Personal Account',
    'B3D__3_1': 'Bank Account: Business Account'
})
df.head()

,county,Rural/Urban,Gender,Language,Education,Mobile Money: Personal Account,Bank Account: Business Account
0,Garissa,Urban,Male,English,"""Post Secondary education",#NULL!,#NULL!
1,Garissa,Urban,Male,English,Upper primary,#NULL!,#NULL!
2,Busia,Urban,Female,Swahili,Upper primary,#NULL!,#NULL!
3,Kiambu,Urban,Male,English,"""Secondary/High school",#NULL!,#NULL!
4,Murang'a,Urban,Female,Swahili,"""Post Secondary education",#NULL!,#NULL!


In [23]:
missing_values = df.isnull().sum()
print(missing_values)

county                               0
Rural/Urban                          0
Gender                               0
Language                             0
Education                         3013
Mobile Money: Personal Account       0
Bank Account: Business Account       0
dtype: int64


#### Education column has 3013 missing values out of 20871. Fill these missing values with the mode. 

In [31]:
education_mode = df['Education'].mode()[0]
df['Education'] = df['Education'].fillna(education_mode)
df['Education'].isnull().sum()

0

In [33]:
missing_values = df.isnull().sum()
print(missing_values)

county                            0
Rural/Urban                       0
Gender                            0
Language                          0
Education                         0
Mobile Money: Personal Account    0
Bank Account: Business Account    0
dtype: int64


In [42]:
print(df.dtypes)

county                            object
Rural/Urban                       object
Gender                            object
Language                          object
Education                         object
Mobile Money: Personal Account    object
Bank Account: Business Account    object
dtype: object
